In [ ]:
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

# Define the extracted features
def extract_statistical_features(data):
    features = [
        np.mean(data),
        np.median(data),
        np.quantile(data, 0.25),
        np.quantile(data, 0.75),
        np.quantile(data, 0.75) - np.quantile(data, 0.25),  # IQR
        np.std(data),
        np.var(data),
        skew(data),
        kurtosis(data),
        np.sum(data**2) / len(data)  # 能量
    ]
    return features

# normalization (masked region)
def min_max_normalization(image_array, mask_array):
    masked_data = image_array[mask_array > 0]
    min_val = np.min(masked_data)
    max_val = np.max(masked_data)
    normalized_array = (image_array - min_val) / (max_val - min_val) 
    return normalized_array

# extract features from MRI and entropy map
def process_superpixels(mri_image_path, entropy_image_path, superpixel_mask_path, output_dir):
        # load image
        mri_image = sitk.ReadImage(mri_image_path)
        entropy_image = sitk.ReadImage(entropy_image_path)
        superpixel_mask = sitk.ReadImage(superpixel_mask_path)

        # Convert images to numpy arrays and downcast to float32
        mri_array = sitk.GetArrayFromImage(mri_image).astype(np.float32)
        entropy_array = sitk.GetArrayFromImage(entropy_image).astype(np.float32)
        superpixel_mask_array = sitk.GetArrayFromImage(superpixel_mask).astype(np.int32)

        # normalization (masked region)
        mri_array_normalized = min_max_normalization(mri_array, superpixel_mask_array)
        entropy_array_normalized = min_max_normalization(entropy_array, superpixel_mask_array)

        # Initialize a list to store the features of each supervoxel
        features_list = []
        labels_list = []

        # Extracting superpixel labels
        unique_labels = np.unique(superpixel_mask_array)
        for label in unique_labels:
            if label == 0:  # Skip Background
                continue
            mask = (superpixel_mask_array == label)
            mri_data = mri_array_normalized[mask]
            entropy_data = entropy_array_normalized[mask]

            # feature extraction
            mri_features = extract_statistical_features(mri_data)
            entropy_features = extract_statistical_features(entropy_data)
            
            # Combining all features and labels
            combined_features = mri_features + entropy_features
            features_list.append(combined_features)
            labels_list.append(label)

        # Define feature name
        columns = ['MRI_mean', 'MRI_median', 'MRI_q1', 'MRI_q3', 'MRI_iqr', 'MRI_std', 'MRI_variance', 'MRI_skewness', 'MRI_kurtosis', 'MRI_energy',
                   'Entropy_mean', 'Entropy_median', 'Entropy_q1', 'Entropy_q3', 'Entropy_iqr', 'Entropy_std', 'Entropy_variance', 'Entropy_skewness', 'Entropy_kurtosis', 'Entropy_energy']

        # sava file
        features_df = pd.DataFrame(features_list, index=labels_list, columns=columns)
        sample_number = os.path.basename(mri_image_path)
        features_file_path = os.path.join(output_dir, f'{sample_number}.csv')
        features_df.to_csv(features_file_path, index_label='Superpixel_Label')

# example usage
mri_image_dir = 'File path'
entropy_image_dir = 'File path'
superpixel_mask_dir = 'File path'
output_dir = 'File path'

process_superpixels(mri_image_dir, entropy_image_dir, superpixel_mask_dir, output_dir)
